In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode()
import warnings
warnings.filterwarnings("ignore")

In [ ]:
headcount = pd.read_excel('../input/Demandv1.1.xlsx',sheet_name='Headcount')
# billable data shows actual demand of past years
headcount = headcount[headcount.Status == 'Billable']

In [ ]:
# converting Technologies into one hot encoding
from re import split
cleaned = headcount.set_index('Employee Code').SkillList.str.split(r',\s*(?![^()]*\))', expand=True).stack()
one_hot_coded_df = pd.get_dummies(cleaned).groupby(level=0).sum()
one_hot_coded_df.head()

In [ ]:
# making index as Employee code for main data frame
head_df = pd.read_excel('../input/Demandv1.1.xlsx',sheet_name='Headcount',index_col='Employee Code')

In [ ]:
# merging to daat frame
merged_df = pd.merge(head_df,one_hot_coded_df,left_index=True,right_index=True)

In [ ]:
# removing index and unwanted columns
removed_index = merged_df.reset_index(drop=True)

In [ ]:
removed_index.columns

In [ ]:
final_df = removed_index.drop(['Region','  Last Name','Status','Market Unit','SkillList'],axis=1).set_index('Local Date of Joining')

In [ ]:
final_df['year'] = final_df.index.year

In [ ]:
final_df['month']=final_df.index.month

In [ ]:
grouped_df = final_df.reset_index(drop=True).groupby(['year','month','Location','Designation']).sum()

In [ ]:
grouped_df.head()

In [ ]:
def predict_loc_tech(location,technology,desgination,changepoint_prior):
    if location is None and desgination is None:
        location_df = grouped_df
    elif location is None and desgination:
        location_df = grouped_df[(grouped_df.index.get_level_values('Designation') == desgination)]
    elif desgination is None and location:
        location_df = grouped_df[(grouped_df.index.get_level_values('Location') == location)]
    elif location and desgination:
        location_df = grouped_df[(grouped_df.index.get_level_values('Location') == location) & (grouped_df.index.get_level_values('Designation') == desgination)]
    location_df['day'] = 1
    location_df = location_df.reset_index()
    location_df = location_df[location_df.year >= 2010]
    location_df['Date']=pd.to_datetime(location_df[['year','month','day']])
    loc_tech_df = location_df.set_index('Date').resample('M').sum()[[technology]]
    loc_tech_df.apply(np.log1p).plot()
    plt.show()
    loc_tech_df = loc_tech_df.apply(np.log1p)
    loc_tech_prophet = loc_tech_df.reset_index().rename(columns={'Date':'ds',technology:'y'})
    
    from fbprophet import Prophet
    model = Prophet(changepoint_prior_scale=changepoint_prior,yearly_seasonality=20) #instantiate Prophet
    model.fit(loc_tech_prophet.loc[:87]); #fit the model with your dataframe
    
    future = model.make_future_dataframe(periods=12,freq='M')
    forecast = model.predict(future)
    print('RMSE: %f' % np.sqrt(np.mean(((np.round(np.exp(forecast['yhat'].loc[87:])-1))-(np.exp(loc_tech_prophet['y'].loc[87:])-1))**2)))
#     print(future)
#     print(forecast.loc[88:])
#     print(loc_tech_prophet.loc[88:])

    py.iplot([
        go.Scatter(x=loc_tech_prophet['ds'], y=loc_tech_prophet['y'], name='Actual'),
        go.Scatter(x=forecast['ds'], y=forecast['yhat'], name='Predicted'),
        go.Scatter(x=forecast['ds'], y=forecast['yhat_upper'], fill='tonexty', mode='none', name='upper'),
        go.Scatter(x=forecast['ds'], y=forecast['yhat_lower'], fill='tonexty', mode='none', name='lower'),
        go.Scatter(x=forecast['ds'], y=forecast['trend'], name='Trend')
    ])
    
    model.plot(forecast)
    model.plot_components(forecast)

In [ ]:
predict_loc_tech('Pune','Java',None,10)

In [ ]:
def forecast_next_year(location,technology,desgination,changepoint_prior):
    if location is None and desgination is None:
        location_df = grouped_df
    elif location is None and desgination:
        location_df = grouped_df[(grouped_df.index.get_level_values('Designation') == desgination)]
    elif desgination is None and location:
        location_df = grouped_df[(grouped_df.index.get_level_values('Location') == location)]
    elif location and desgination:
        location_df = grouped_df[(grouped_df.index.get_level_values('Location') == location) & (grouped_df.index.get_level_values('Designation') == desgination)]
    location_df['day'] = 1
    location_df = location_df.reset_index()
    location_df = location_df[location_df.year >= 2010]
    location_df['Date']=pd.to_datetime(location_df[['year','month','day']])
    loc_tech_df = location_df.set_index('Date').resample('M').sum()[[technology]]
    loc_tech_df = loc_tech_df.apply(np.log1p)
    loc_tech_prophet = loc_tech_df.reset_index().rename(columns={'Date':'ds',technology:'y'})
    
    from fbprophet import Prophet
    model = Prophet(changepoint_prior_scale=changepoint_prior,yearly_seasonality=20) #instantiate Prophet
    model.fit(loc_tech_prophet); #fit the model with your dataframe
    
    future = model.make_future_dataframe(periods=12,freq='M')
    forecast = model.predict(future)
    print('RMSE over all data points: %f' % np.sqrt(np.mean(((np.round(np.exp(forecast['yhat'].iloc[:-12])-1))-(np.exp(loc_tech_prophet['y'])-1))**2)))
    return forecast

In [ ]:
forecasted_values =forecast_next_year('Pune','Java',None,13)

****Prediction for next 12 months****

In [ ]:
forecasted_values.iloc[-12:]['yhat']